<a href="https://colab.research.google.com/github/ShrawanSai/Automated-ICD10-Medical-Coding-/blob/master/Sentence_Classification_with_Bert_%26_Ernie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/brunneis/ernie/raw/master/misc/ernie-sticker-diecut.png" height="150px" alt="Ernie Logo" />

In [12]:
%tensorflow_version 2.x
!pip install ernie==0.0.27b0 > /dev/null

In [13]:
from ernie import SentenceClassifier, Models
import pandas as pd

# Binary Classification

In [ ]:
tuples = [("This is a negative sentence. Everything was wrong today at work.", 0),
          ("This is a positive example. I'm very happy today.", 1)]
df = pd.DataFrame(tuples)

classifier = SentenceClassifier(model_name=Models.RobertaBaseCased, max_length=64)
classifier.load_dataset(df, validation_split=0.2)
classifier.fine_tune(epochs=4, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)

sentence = "Oh, that's great!"
probabilities = classifier.predict_one(sentence)
print(probabilities)

2020-04-13 11:12:15 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-04-13 11:12:16 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-04-13 11:12:22 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


2020-04-13 11:12:23 [WARNING] Gradients do not exist for variables ['tf_roberta_for_sequence_classification/roberta/pooler/dense/kernel:0', 'tf_roberta_for_sequence_classification/roberta/pooler/dense/bias:0'] when minimizing the loss.


1/1 [==============================] - 0s 41ms/step - loss: 0.4277 - accuracy: 1.0000 - val_loss: 1.1621 - val_accuracy: 0.0000e+00
(0.37437606819563046, 0.6256239318043696)


# Multi-Class Classification

In [3]:
tuples = [("This is a negative sentence. Everything was wrong today at work.", 0),
          ("This is a positive example. I'm very happy today.", 1),
          ("This is a neutral sentence. That's normal.", 2)]
df = pd.DataFrame(tuples)

classifier = SentenceClassifier(model_name=Models.AlbertLargeCased2, max_length=64, labels_no=3)
classifier.load_dataset(df, validation_split=0.2)
classifier.fine_tune(epochs=4, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)

sentence = "Oh, that's great!"
probabilities = classifier.predict_one(sentence)
print(probabilities)


2/2 [==============================] - 0s 221ms/step - loss: 0.1092 - accuracy: 1.0000 - val_loss: 4.3492 - val_accuracy: 0.0000e+00
(0.2738850812887233, 0.7121831805617821, 0.013931738149494692)


In [14]:
import re
from sklearn.preprocessing import LabelEncoder

In [15]:
file = pd.read_csv('codes.csv',header =None)
file.columns =  ['desc','codes']
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
file['encodes'] = labelencoder.fit_transform(file['codes'])
file.head()

,desc,codes,encodes
0,"Cholera due to Vibrio cholerae 01, biovar chol...",A00,0
1,"Cholera due to Vibrio cholerae 01, biovar eltor",A00,0
2,"Cholera, unspecified",A00,0
3,"Typhoid fever, unspecified",A01,1
4,Typhoid meningitis,A01,1


In [16]:
file = file[['desc','encodes']]

In [17]:
file.head()

,desc,encodes
0,"Cholera due to Vibrio cholerae 01, biovar chol...",0
1,"Cholera due to Vibrio cholerae 01, biovar eltor",0
2,"Cholera, unspecified",0
3,"Typhoid fever, unspecified",1
4,Typhoid meningitis,1


In [ ]:
classifier = SentenceClassifier(model_name=Models.AlbertLargeCased2, max_length=64, labels_no=966)
classifier.load_dataset(file, validation_split=0.2)
classifier.fine_tune(epochs=4, learning_rate=2e-5, training_batch_size=32, validation_batch_size=64)

233/374 [=================>............] - ETA: 5:14 - loss: 6.2934 - accuracy: 0.0174

In [ ]:
classifier.dump('./model')

# Interrupted Training
Since the execution may break during training (especially if you are using Google Colab), you can opt to secure every new trained epoch, so the training can be resumed without losing all the progress.

In [ ]:
tuples = [("This is a negative sentence. Everything was wrong today at work.", 0),
          ("This is a positive example. I'm very happy today.", 1)]
df = pd.DataFrame(tuples)

classifier = SentenceClassifier(model_name=Models.BertBaseUncased, max_length=64)
classifier.load_dataset(df, validation_split=0.2)

In [ ]:
for epoch in range(1, 5):
  if epoch == 3:
    raise Exception("Forced crash")

  classifier.fine_tune(epochs=1)
  classifier.dump(f'./my-model/{epoch}')

1/1 [==============================] - 2s 2s/step - loss: 0.7122 - accuracy: 0.0000e+00 - val_loss: 0.6077 - val_accuracy: 1.0000


Exception: ignored

In [ ]:
!ls ./my-model/*

./my-model/1:
config.json		 tf_model.h5		vocab.txt
special_tokens_map.json  tokenizer_config.json

./my-model/2:
config.json		 tf_model.h5		vocab.txt
special_tokens_map.json  tokenizer_config.json


In [ ]:
last_training_epoch = 2

classifier = SentenceClassifier(model_path=f'./my-model/{last_training_epoch}')
classifier.load_dataset(df, validation_split=0.2)

for epoch in range(last_training_epoch + 1, 5):
  classifier.fine_tune(epochs=1)
  classifier.dump(f'./my-model/{epoch}')

1/1 [==============================] - 2s 2s/step - loss: 0.8209 - accuracy: 0.0000e+00 - val_loss: 0.6844 - val_accuracy: 1.0000


# Autosave
Even if you do not explicitly dump the model, it will be autosaved into `./ernie-autosave` every time `fine_tune` is successfully executed. 

```
ernie-autosave/
└── model_family/
    └── timestamp/
        ├── config.json
        ├── special_tokens_map.json
        ├── tf_model.h5
        ├── tokenizer_config.json
        └── vocab.txt
```

In [ ]:
!ls -R ./ernie-autosave

./ernie-autosave:
albert	bert  roberta

./ernie-autosave/albert:
1586776396535

./ernie-autosave/albert/1586776396535:
config.json		 spiece.model  tokenizer_config.json
special_tokens_map.json  tf_model.h5

./ernie-autosave/bert:
1586776442341  1586776466469  1586777096444  1586777121356

./ernie-autosave/bert/1586776442341:
config.json		 tf_model.h5		vocab.txt
special_tokens_map.json  tokenizer_config.json

./ernie-autosave/bert/1586776466469:
config.json		 tf_model.h5		vocab.txt
special_tokens_map.json  tokenizer_config.json

./ernie-autosave/bert/1586777096444:
config.json		 tf_model.h5		vocab.txt
special_tokens_map.json  tokenizer_config.json

./ernie-autosave/bert/1586777121356:
config.json		 tf_model.h5		vocab.txt
special_tokens_map.json  tokenizer_config.json

./ernie-autosave/roberta:
1586776352887

./ernie-autosave/roberta/1586776352887:
config.json  special_tokens_map.json  tokenizer_config.json
merges.txt   tf_model.h5	      vocab.json


You can easily clean the autosaved models by invoking `clean_autosave` after finishing a session or when starting a new one.

In [ ]:
from ernie import clean_autosave
clean_autosave()

In [ ]:
!ls -R ./ernie-autosave

ls: cannot access './ernie-autosave': No such file or directory
